# Title
[]()
Copied code from https://github.com/mukulpatnaik/researchgpt/blob/main/main.py

In [ ]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [1]:

from PyPDF2 import PdfReader
from openai.embeddings_utils import get_embedding, cosine_similarity

In [2]:
# from flask import Flask, request, render_template
from io import BytesIO
import pandas as pd
import openai
import os
import requests
# from flask_cors import CORS
# from _md5 import md5
# from google.cloud import storage


In [5]:
import json
with open("../portfolio-projects/online-PT-social-media-NLP/notebooks/credentials.json") as f:
    credentials = json.load(f)

openai.api_key = credentials['OPENAI_API_KEY']

In [7]:
class Chatbot():
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page = pdf.pages[i]
            page_text = []

            def visitor_body(text, cm, tm, fontDict, fontSize):
                x = tm[4]
                y = tm[5]
                # ignore header/footer
                if (y > 50 and y < 720) and (len(text.strip()) > 1):
                    page_text.append({
                    'fontsize': fontSize,
                    'text': text.strip().replace('\x03', ''),
                    'x': x,
                    'y': y
                    })

            # _ = page.extract_text(visitor_text=visitor_body)

            blob_font_size = None
            blob_text = ''
            processed_text = []

            for t in page_text:
                if t['fontsize'] == blob_font_size:
                    blob_text += f" {t['text']}"
                    if len(blob_text) >= 2000:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                        blob_font_size = None
                        blob_text = ''
                else:
                    if blob_font_size is not None and len(blob_text) >= 1:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                    blob_font_size = t['fontsize']
                    blob_text = t['text']
                paper_text += processed_text
        print("Done parsing paper")
        # print(paper_text)
        return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 30:
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df

    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv('OPENAI_API_KEY')
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        print(result)
        system_role = """whose expertise is reading and summarizing scientific papers. You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the languange of the query: 
            
        Here is the question: """+ user_input + """
            
        and here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        openai.api_key = os.getenv('OPENAI_API_KEY')
        r = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, max_tokens=1500)
        answer = r.choices[0]["message"]["content"]
        print('Done sending request to GPT-3')
        response = {'answer': answer, 'sources': sources}
        return response

In [9]:
# Custom
def process_pdf(filename, filepath):
    print("Processing pdf")
    filename = f'{filepath}/'.replace('\\','/')+filename

    # key = md5(filename).hexdigest()
    # print(key)
    # # Create a Cloud Storage client.
    # gcs = storage.Client()
    # name = key+'.json'

    # # Get the bucket that the file will be uploaded to.
    # bucket = gcs.get_bucket(CLOUD_STORAGE_BUCKET)
    # # Check if the file already exists
    # if bucket.blob(name).exists():
    #     print("File already exists")
    #     print("Done processing pdf")
    #     return {"key": key}

    # pdf = PdfReader(BytesIO(filename))
    pdf = PdfReader(filename)
    chatbot = Chatbot()
    paper_text = chatbot.extract_text(pdf)
    # df = chatbot.create_df(paper_text)
    # df = chatbot.embeddings(df)
    
    # # # Create a new blob and upload the file's content.
    # # blob = bucket.blob(name)
    # # blob.upload_from_string(df.to_json(), content_type='application/json')
    # # # if db.get(key) is None:
    # # #     db.set(key, df.to_json())
    # # print("Done processing pdf")
    # # return {"key": key}
    # return df, pdf
    return paper_text, pdf

filename = "Hypohydration but not menstrual phase influences pain perception in healthy women.pdf"
filepath = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"

df, pdf = process_pdf(filename, filepath)

Processing pdf
Parsing paper
Total number of pages: 27
Done parsing paper


In [12]:
# Custom
def process_pdf(filename, filepath):
    print("Processing pdf")
    filename = f'{filepath}/'.replace('\\','/')+filename

    # key = md5(filename).hexdigest()
    # print(key)
    # # Create a Cloud Storage client.
    # gcs = storage.Client()
    # name = key+'.json'

    # # Get the bucket that the file will be uploaded to.
    # bucket = gcs.get_bucket(CLOUD_STORAGE_BUCKET)
    # # Check if the file already exists
    # if bucket.blob(name).exists():
    #     print("File already exists")
    #     print("Done processing pdf")
    #     return {"key": key}

    # pdf = PdfReader(BytesIO(filename))
    pdf = PdfReader(filename)
    # chatbot = Chatbot()
    # paper_text = chatbot.extract_text(pdf)
    # # df = chatbot.create_df(paper_text)
    # # df = chatbot.embeddings(df)
    
    # # # # Create a new blob and upload the file's content.
    # # # blob = bucket.blob(name)
    # # # blob.upload_from_string(df.to_json(), content_type='application/json')
    # # # # if db.get(key) is None:
    # # # #     db.set(key, df.to_json())
    # # # print("Done processing pdf")
    # # # return {"key": key}
    # # return df, pdf
    # return paper_text, pdf
    return pdf

filename = "Hypohydration but not menstrual phase influences pain perception in healthy women.pdf"
filepath = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"

pdf = process_pdf(filename, filepath)

Processing pdf


In [19]:
pdf.pages[0].extract_text()

'Hypohydration but not Menstrual Phase Influences Pain Perceptio n in Healthy Women 1 Beverly Tan\n1, Michael C. Philipp2, Ahmad Munir Che Muhamed3, Toby Mündel1* 2 \n1 Massey University, School of Sport Exercise and Nutrition, Palmerston North, New Zealand 3 \n2 Massey University, School of Psychology, Palmerston North, New Zealand 4 \n3 Advanced Medical and Dental Institute, Universiti Sains Malaysi a, Pulau Pinang, Malaysia 5  6 \n*Corresponding Author 7 Toby Mündel: T.Mundel@massey.ac.nz \n8 School of Sport Exercise and Nutrition (PN621) 9 Massey University  10 Private Bag 11 222 11 Palmerston North 4442 12 New Zealand 13  14 ORCID:  \n15 Ahmad Munir Che Muhamed: 0000-0002-0254-6838 16 Michael Philipp: 0000-0001-8203-8018 17 Toby Mündel: 0000-0002-4214-8543 18 \n 19 Running title:  \n20 Hydration, Menstrual Phase and Pain\n  21 '

## Debug 2

In [37]:
class Chatbot():
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page_text = pdf.pages[i].extract_text()
            paper_text.append(page_text)
        return paper_text


        #     def visitor_body(text, cm, tm, fontDict, fontSize):
        #         x = tm[4]
        #         y = tm[5]
        #         # ignore header/footer
        #         if (y > 50 and y < 720) and (len(text.strip()) > 1):
        #             page_text.append({
        #             'fontsize': fontSize,
        #             'text': text.strip().replace('\x03', ''),
        #             'x': x,
        #             'y': y
        #             })

        #     # _ = page.extract_text(visitor_text=visitor_body)

        #     blob_font_size = None
        #     blob_text = ''
        #     processed_text = []

        #     for t in page_text:
        #         if t['fontsize'] == blob_font_size:
        #             blob_text += f" {t['text']}"
        #             if len(blob_text) >= 2000:
        #                 processed_text.append({
        #                     'fontsize': blob_font_size,
        #                     'text': blob_text,
        #                     'page': i
        #                 })
        #                 blob_font_size = None
        #                 blob_text = ''
        #         else:
        #             if blob_font_size is not None and len(blob_text) >= 1:
        #                 processed_text.append({
        #                     'fontsize': blob_font_size,
        #                     'text': blob_text,
        #                     'page': i
        #                 })
        #             blob_font_size = t['fontsize']
        #             blob_text = t['text']
        #         paper_text += processed_text
        # print("Done parsing paper")
        # # print(paper_text)
        # return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 30:
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df

    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv('OPENAI_API_KEY')
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        print(result)
        system_role = """whose expertise is reading and summarizing scientific papers. You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the languange of the query: 
            
        Here is the question: """+ user_input + """
            
        and here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        openai.api_key = os.getenv('OPENAI_API_KEY')
        r = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, max_tokens=1500)
        answer = r.choices[0]["message"]["content"]
        print('Done sending request to GPT-3')
        response = {'answer': answer, 'sources': sources}
        return response

In [42]:
# Custom
def process_pdf(filename, filepath):
    print("Processing pdf")
    filename = f'{filepath}/'.replace('\\','/')+filename

    # key = md5(filename).hexdigest()
    # print(key)
    # # Create a Cloud Storage client.
    # gcs = storage.Client()
    # name = key+'.json'

    # # Get the bucket that the file will be uploaded to.
    # bucket = gcs.get_bucket(CLOUD_STORAGE_BUCKET)
    # # Check if the file already exists
    # if bucket.blob(name).exists():
    #     print("File already exists")
    #     print("Done processing pdf")
    #     return {"key": key}

    # pdf = PdfReader(BytesIO(filename))
    pdf = PdfReader(filename)
    chatbot = Chatbot()
    paper_text = chatbot.extract_text(pdf)
    df = chatbot.create_df(paper_text)
    df = chatbot.embeddings(df)
    
    # # Create a new blob and upload the file's content.
    # blob = bucket.blob(name)
    # blob.upload_from_string(df.to_json(), content_type='application/json')
    # # if db.get(key) is None:
    # #     db.set(key, df.to_json())
    # print("Done processing pdf")
    # return {"key": key}
    return df, pdf
    # return paper_text, pdf

filename = "Hypohydration but not menstrual phase influences pain perception in healthy women.pdf"
filepath = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"

df2, pdf2 = process_pdf(filename, filepath)


Processing pdf
Parsing paper
Total number of pages: 27
Creating dataframe


TypeError: string indices must be integers

In [39]:
len(df2)

27

In [40]:
for row in df2:
    print(row)

Hypohydration but not Menstrual Phase Influences Pain Perceptio n in Healthy Women 1 Beverly Tan
1, Michael C. Philipp2, Ahmad Munir Che Muhamed3, Toby Mündel1* 2 
1 Massey University, School of Sport Exercise and Nutrition, Palmerston North, New Zealand 3 
2 Massey University, School of Psychology, Palmerston North, New Zealand 4 
3 Advanced Medical and Dental Institute, Universiti Sains Malaysi a, Pulau Pinang, Malaysia 5  6 
*Corresponding Author 7 Toby Mündel: T.Mundel@massey.ac.nz 
8 School of Sport Exercise and Nutrition (PN621) 9 Massey University  10 Private Bag 11 222 11 Palmerston North 4442 12 New Zealand 13  14 ORCID:  
15 Ahmad Munir Che Muhamed: 0000-0002-0254-6838 16 Michael Philipp: 0000-0001-8203-8018 17 Toby Mündel: 0000-0002-4214-8543 18 
 19 Running title:  
20 Hydration, Menstrual Phase and Pain
  21 
ABSTRACT (239 words)  22 Chronic pain is a pervasive health problem and is associated wi th tremendous socioeconomic 23 costs. However, current pain treatments are of

## Debug 3

In [45]:
class Chatbot():
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page_text = pdf.pages[i].extract_text().strip().replace('\x03', '')
            paper_text.append({
                'text': page_text,
                'page': i
                })
        return paper_text


        #     def visitor_body(text, cm, tm, fontDict, fontSize):
        #         x = tm[4]
        #         y = tm[5]
        #         # ignore header/footer
        #         if (y > 50 and y < 720) and (len(text.strip()) > 1):
        #             page_text.append({
        #             'fontsize': fontSize,
        #             'text': text.strip().replace('\x03', ''),
        #             'x': x,
        #             'y': y
        #             })

        #     # _ = page.extract_text(visitor_text=visitor_body)

        #     blob_font_size = None
        #     blob_text = ''
        #     processed_text = []

        #     for t in page_text:
        #         if t['fontsize'] == blob_font_size:
        #             blob_text += f" {t['text']}"
        #             if len(blob_text) >= 2000:
        #                 processed_text.append({
        #                     'fontsize': blob_font_size,
        #                     'text': blob_text,
        #                     'page': i
        #                 })
        #                 blob_font_size = None
        #                 blob_text = ''
        #         else:
        #             if blob_font_size is not None and len(blob_text) >= 1:
        #                 processed_text.append({
        #                     'fontsize': blob_font_size,
        #                     'text': blob_text,
        #                     'page': i
        #                 })
        #             blob_font_size = t['fontsize']
        #             blob_text = t['text']
        #         paper_text += processed_text
        # print("Done parsing paper")
        # # print(paper_text)
        # return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 30:
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df

    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv('OPENAI_API_KEY')
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        print(result)
        system_role = """whose expertise is reading and summarizing scientific papers. You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the languange of the query: 
            
        Here is the question: """+ user_input + """
            
        and here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        openai.api_key = os.getenv('OPENAI_API_KEY')
        r = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, max_tokens=1500)
        answer = r.choices[0]["message"]["content"]
        print('Done sending request to GPT-3')
        response = {'answer': answer, 'sources': sources}
        return response

In [46]:
# Custom
def process_pdf(filename, filepath):
    print("Processing pdf")
    filename = f'{filepath}/'.replace('\\','/')+filename

    # key = md5(filename).hexdigest()
    # print(key)
    # # Create a Cloud Storage client.
    # gcs = storage.Client()
    # name = key+'.json'

    # # Get the bucket that the file will be uploaded to.
    # bucket = gcs.get_bucket(CLOUD_STORAGE_BUCKET)
    # # Check if the file already exists
    # if bucket.blob(name).exists():
    #     print("File already exists")
    #     print("Done processing pdf")
    #     return {"key": key}

    # pdf = PdfReader(BytesIO(filename))
    pdf = PdfReader(filename)
    chatbot = Chatbot()
    paper_text = chatbot.extract_text(pdf)
    df = chatbot.create_df(paper_text)
    df = chatbot.embeddings(df)
    
    # # Create a new blob and upload the file's content.
    # blob = bucket.blob(name)
    # blob.upload_from_string(df.to_json(), content_type='application/json')
    # # if db.get(key) is None:
    # #     db.set(key, df.to_json())
    # print("Done processing pdf")
    # return {"key": key}
    return df, pdf
    # return paper_text, pdf

filename = "Hypohydration but not menstrual phase influences pain perception in healthy women.pdf"
filepath = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"

df3, pdf3 = process_pdf(filename, filepath)


Processing pdf
Parsing paper
Total number of pages: 27
Creating dataframe
Done creating dataframe
Calculating embeddings


RetryError: RetryError[<Future at 0x2279ce30b20 state=finished raised AuthenticationError>]

## Debug 4

In [70]:
class Chatbot():
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page_text = pdf.pages[i].extract_text().strip().replace('\x03', '')
            paper_text.append({
                'text': page_text,
                'page': i
                })
        return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 30:
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df

    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv('api_openai')
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        print(result)
        system_role = """whose expertise is reading and summarizing scientific papers. You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the languange of the query: 
            
        Here is the question: """+ user_input + """
            
        and here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        openai.api_key = os.getenv('api_openai')
        r = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, max_tokens=1500)
        answer = r.choices[0]["message"]["content"]
        print('Done sending request to GPT-3')
        response = {'answer': answer, 'sources': sources}
        return response

In [56]:
# Custom
def process_pdf(filename, filepath):
    print("Processing pdf")
    filename = f'{filepath}/'.replace('\\','/')+filename

    pdf = PdfReader(filename)
    chatbot = Chatbot()
    paper_text = chatbot.extract_text(pdf)
    df = chatbot.create_df(paper_text)
    df = chatbot.embeddings(df)
    
    # # Create a new blob and upload the file's content.
    # blob = bucket.blob(name)
    # blob.upload_from_string(df.to_json(), content_type='application/json')
    # # if db.get(key) is None:
    # #     db.set(key, df.to_json())
    # print("Done processing pdf")
    # return {"key": key}
    return df, pdf
    # return paper_text, pdf

filename = "Hypohydration but not menstrual phase influences pain perception in healthy women.pdf"
filepath = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"

df4, pdf4 = process_pdf(filename, filepath)


Processing pdf
Parsing paper
Total number of pages: 27
Creating dataframe
Done creating dataframe
Calculating embeddings
Done calculating embeddings


In [57]:
df4

,text,page,length,embeddings
0,Hypohydration but not Menstrual Phase Influenc...,0,832,"[-0.012927941046655178, -0.00770729873329401, ..."
1,ABSTRACT (239 words) 22 Chronic pain is a per...,1,2286,"[-0.022682175040245056, -0.007203123066574335,..."
2,INTRODUCTION 50 Pain is recognized as a public...,2,4064,"[-0.01519670058041811, -0.007850742898881435, ..."
3,confounded previous research on effects of the...,3,1001,"[-0.02789771743118763, -0.004251896869391203, ..."
4,METHODS 105 Ethical Approval 106 The study was...,4,3597,"[-0.025527486577630043, -0.006997073534876108,..."
5,(see Maximal Handgrip Strength Assessment) to ...,5,3750,"[-0.03222351893782616, 0.0027854228392243385, ..."
6,"Twenty-four hours prior to each trial, partici...",6,4069,"[-0.011679299175739288, 0.0027374434284865856,..."
7,"intensity and unpleasantness again, to verify ...",7,3324,"[-0.017357170581817627, -0.01086043007671833, ..."
8,were assessed to ensure they were normally dis...,8,1565,"[-0.014132192358374596, 0.014319879934191704, ..."
9,RESULTS 292 Following analysis of ovarian horm...,9,3641,"[-0.020886564627289772, -0.008561731316149235,..."


In [58]:
df4.loc[0,'text']

'Hypohydration but not Menstrual Phase Influences Pain Perceptio n in Healthy Women 1 Beverly Tan\n1, Michael C. Philipp2, Ahmad Munir Che Muhamed3, Toby Mündel1* 2 \n1 Massey University, School of Sport Exercise and Nutrition, Palmerston North, New Zealand 3 \n2 Massey University, School of Psychology, Palmerston North, New Zealand 4 \n3 Advanced Medical and Dental Institute, Universiti Sains Malaysi a, Pulau Pinang, Malaysia 5  6 \n*Corresponding Author 7 Toby Mündel: T.Mundel@massey.ac.nz \n8 School of Sport Exercise and Nutrition (PN621) 9 Massey University  10 Private Bag 11 222 11 Palmerston North 4442 12 New Zealand 13  14 ORCID:  \n15 Ahmad Munir Che Muhamed: 0000-0002-0254-6838 16 Michael Philipp: 0000-0001-8203-8018 17 Toby Mündel: 0000-0002-4214-8543 18 \n 19 Running title:  \n20 Hydration, Menstrual Phase and Pain\n  21'

In [72]:
def reply(df, query):
    chatbot = Chatbot()
    prompt = chatbot.create_prompt(df, query)
    response = chatbot.gpt(prompt)
    print(response)
    return response


In [60]:

query = """
What is the favourite destination in Vietnam amongst tourists from Canada?
"""

reply4 = reply(df4, query)

[{'Page 15': 'Competing Interests and Funding 511 No potential conflicts or competing interests are disclosed. No external funding was received.  512 Author Contrib...'}, {'Page 6': 'Twenty-four hours prior to each trial, participants were instru cted to either restrict fluid intake 190 and avoid consumption of high-water content f...'}, {'Page 18': '37. Sawka MN, Burke LM, Eichner ER, Maughan RJ, Montain SJ & Stache nfeld NS. (2007). 611 American College of Sports Medicine position stand. Exercise...'}]
                                                text  page  length  \
0  Competing Interests and Funding 511 No potenti...    15     614   
1  Twenty-four hours prior to each trial, partici...     6    4069   
2  37. Sawka MN, Burke LM, Eichner ER, Maughan RJ...    18    3596   

                                          embeddings  similarity  
0  [-0.03124002367258072, -0.007716950494796038, ...    0.692408  
1  [-0.011679299175739288, 0.0027374434284865856,...    0.683260  
2  [-0.0

In [61]:
reply

{'answer': 'I apologize, but the provided embeddings do not seem to be relevant to the question about the favorite destination in Vietnam for Canadian tourists. The embeddings seem to be related to a study on the effects of hypohydration on pain perception and a list of references from different scientific papers. Can you please provide me with different embeddings that are relevant to your query?',
 'sources': [{'Page 15': 'Competing Interests and Funding 511 No potential conflicts or competing interests are disclosed. No external funding was received.  512 Author Contrib...'},
  {'Page 6': 'Twenty-four hours prior to each trial, participants were instru cted to either restrict fluid intake 190 and avoid consumption of high-water content f...'},
  {'Page 18': '37. Sawka MN, Burke LM, Eichner ER, Maughan RJ, Montain SJ & Stache nfeld NS. (2007). 611 American College of Sports Medicine position stand. Exercise...'}]}

In [68]:
del reply

In [73]:
query2 = """
How does menstrual cycle affect pain?
"""

reply4_2 = reply(df4, query2)
reply4_2

[{'Page 16': 'REFERENCES  521 1. Goldberg DS & McGee SJ. (2011). Pain as a global public health priority. BMC Public 522 Health 11, 770. 523 2. Treede RD, Rief W, B...'}, {'Page 19': '55. Straneva PA, Maixner W, Light KC, Pedersen CA, Costello NL & Girdler SS. (2002). 657 Menstrual cycle, beta-endorphins, and pain sensitivity in pre...'}, {'Page 12': 'Menstrual Phase and Experimental Pain Sensitivity  411 Menstrual phase did not modulate the hyperalgesic effect of hyp ohydration, nor did it 412 inde...'}]
                                                text  page  length  \
0  REFERENCES  521 1. Goldberg DS & McGee SJ. (20...    16    3414   
1  55. Straneva PA, Maixner W, Light KC, Pedersen...    19    2953   
2  Menstrual Phase and Experimental Pain Sensitiv...    12    4164   

                                          embeddings  similarity  
0  [-0.005879143252968788, -0.005777269136160612,...    0.856360  
1  [-0.02594314143061638, -0.012814132496714592, ...    0.856309  
2  [-0.

InvalidRequestError: This model's maximum context length is 4097 tokens. However, you requested 4672 tokens (3172 in the messages, 1500 in the completion). Please reduce the length of the messages or completion.

# Iteration 5

In [ ]:
class Chatbot():
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page_text = pdf.pages[i].extract_text().strip().replace('\x03', '')
            paper_text.append({
                'text': page_text,
                'page': i
                })
        return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 30:
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df

    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv('api_openai')
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        print(result)
        system_role = """whose expertise is reading and summarizing scientific papers. You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the languange of the query: 
            
        Here is the question: """+ user_input + """
            
        and here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        openai.api_key = os.getenv('api_openai')
        r = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.7, max_tokens=1500)
        answer = r.choices[0]["message"]["content"]
        print('Done sending request to GPT-3')
        response = {'answer': answer, 'sources': sources}
        return response

In [74]:
# Custom
def process_pdf(filename, filepath):
    print("Processing pdf")
    filename = f'{filepath}/'.replace('\\','/')+filename

    pdf = PdfReader(filename)
    chatbot = Chatbot()
    paper_text = chatbot.extract_text(pdf)
    df = chatbot.create_df(paper_text)
    df = chatbot.embeddings(df)
    
    # # Create a new blob and upload the file's content.
    # blob = bucket.blob(name)
    # blob.upload_from_string(df.to_json(), content_type='application/json')
    # # if db.get(key) is None:
    # #     db.set(key, df.to_json())
    # print("Done processing pdf")
    # return {"key": key}
    return df, pdf
    # return paper_text, pdf

filename = "Hypohydration but not menstrual phase influences pain perception in healthy women.pdf"
filepath = r"C:\Users\silvh\OneDrive\Ginkgo\knowledge library references\journal articles"



In [76]:
def reply(df, query):
    chatbot = Chatbot()
    prompt = chatbot.create_prompt(df, query)
    try:
        response = chatbot.gpt(prompt)
        return response
    except:
        print('**Unable to get response**')
        return prompt

query2 = """
How does menstrual cycle affect pain?
"""

reply4_2 = reply(df4, query2)
reply4_2

[{'Page 16': 'REFERENCES  521 1. Goldberg DS & McGee SJ. (2011). Pain as a global public health priority. BMC Public 522 Health 11, 770. 523 2. Treede RD, Rief W, B...'}, {'Page 19': '55. Straneva PA, Maixner W, Light KC, Pedersen CA, Costello NL & Girdler SS. (2002). 657 Menstrual cycle, beta-endorphins, and pain sensitivity in pre...'}, {'Page 12': 'Menstrual Phase and Experimental Pain Sensitivity  411 Menstrual phase did not modulate the hyperalgesic effect of hyp ohydration, nor did it 412 inde...'}]
                                                text  page  length  \
0  REFERENCES  521 1. Goldberg DS & McGee SJ. (20...    16    3414   
1  55. Straneva PA, Maixner W, Light KC, Pedersen...    19    2953   
2  Menstrual Phase and Experimental Pain Sensitiv...    12    4164   

                                          embeddings  similarity  
0  [-0.005879143252968788, -0.005777269136160612,...    0.856388  
1  [-0.02594314143061638, -0.012814132496714592, ...    0.856333  
2  [-0.

[{'role': 'system',
  'content': "whose expertise is reading and summarizing scientific papers. You are given a query, \n        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. \n        You must take the given embeddings and return a very detailed summary of the paper in the languange of the query: \n            \n        Here is the question: \nHow does menstrual cycle affect pain?\n\n            \n        and here are the embeddings: \n            \n            1.REFERENCES  521 1. Goldberg DS & McGee SJ. (2011). Pain as a global public health priority. BMC Public 522 Health 11, 770. 523 2. Treede RD, Rief W, Barke A, Aziz Q, Bennett MI, Benoliel R, Coh en M, Evers S, Finnerup 524 NB, First MB, Giamberardino MA, Kaasa S, Kosek E, Lavand'homme P, Nicholas M, Perrot 525 S, Scholz J, Schug S, Smith BH, Svensson P, Vlaeyen JWS & Wang SJ. (2015). A 526 classification of chronic pain for ICD-11. Pain 156, 1003-1007. 527 3. Breivik H

In [79]:
for element in reply4_2:
    print(len(element['content']))
    print(element['content'])

11054
whose expertise is reading and summarizing scientific papers. You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the languange of the query: 
            
        Here is the question: 
How does menstrual cycle affect pain?

            
        and here are the embeddings: 
            
            1.REFERENCES  521 1. Goldberg DS & McGee SJ. (2011). Pain as a global public health priority. BMC Public 522 Health 11, 770. 523 2. Treede RD, Rief W, Barke A, Aziz Q, Bennett MI, Benoliel R, Coh en M, Evers S, Finnerup 524 NB, First MB, Giamberardino MA, Kaasa S, Kosek E, Lavand'homme P, Nicholas M, Perrot 525 S, Scholz J, Schug S, Smith BH, Svensson P, Vlaeyen JWS & Wang SJ. (2015). A 526 classification of chronic pain for ICD-11. Pain 156, 1003-1007. 527 3. Breivik H, Collett B, Ventafridda V, Cohen R & 

In [80]:
query3 = """
Does drinking water improve pain tolerance?
"""

reply5 = reply(df4, query3)
reply5

[{'Page 14': 'hypohydration may worsen chronic pain conditions, and the poten tial efficacy of maintaining 500 adequate hydration in alleviating pain symptoms. This...'}, {'Page 13': '(62). Therefore, this sensitisation effect may have masked pote ntial positive effect of acute water 455 ingestion on pain. Future studies that either...'}, {'Page 1': 'ABSTRACT (239 words)  22 Chronic pain is a pervasive health problem and is associated wi th tremendous socioeconomic 23 costs. However, current pain t...'}]
                                                text  page  length  \
0  hypohydration may worsen chronic pain conditio...    14     941   
1  (62). Therefore, this sensitisation effect may...    13    4200   
2  ABSTRACT (239 words)  22 Chronic pain is a per...     1    2286   

                                          embeddings  similarity  
0  [-0.013870196416974068, -0.005862592253834009,...    0.835406  
1  [-0.015339567326009274, -0.0008556118118576705...    0.833350  
2  [-0.0

{'answer': 'The paper examined the effects of mild hypohydration on pain sensitivity in women and whether acute water ingestion could reverse the negative effects of hypohydration. The study found that hypohydration, caused by 24 hours of fluid restriction, increased pain sensitivity in women and reduced pain tolerance. Acute water ingestion did not reduce pain sensitivity, but it did reduce thirst perception. The study suggests that insufficient water intake throughout the day may exacerbate pain perception in healthy women. Therefore, staying well-hydrated throughout the day is important for maintaining pain tolerance. However, more high-quality research is required to accurately determine whether drinking water can improve pain tolerance.',
 'sources': [{'Page 14': 'hypohydration may worsen chronic pain conditions, and the poten tial efficacy of maintaining 500 adequate hydration in alleviating pain symptoms. This...'},
  {'Page 13': '(62). Therefore, this sensitisation effect may h

In [81]:
query4 = """
Can you rephrase that in under 300 characters?
"""

reply5_4 = reply(df4, query4)
reply5_4

[{'Page 15': 'Competing Interests and Funding 511 No potential conflicts or competing interests are disclosed. No external funding was received.  512 Author Contrib...'}, {'Page 6': 'Twenty-four hours prior to each trial, participants were instru cted to either restrict fluid intake 190 and avoid consumption of high-water content f...'}, {'Page 22': '0246810\nEUH HYPO EUH HYPOVAS (cm)Early Follicular\nBaseline After water ingestion0246810\nEUH HYPO EUH HYPOMid-Luteal\nBaseline After water ingestion‡‡\n†...'}]
                                                text  page  length  \
0  Competing Interests and Funding 511 No potenti...    15     614   
1  Twenty-four hours prior to each trial, partici...     6    4069   
2  0246810\nEUH HYPO EUH HYPOVAS (cm)Early Follic...    22     153   

                                          embeddings  similarity  
0  [-0.03124002367258072, -0.007716950494796038, ...    0.723571  
1  [-0.011679299175739288, 0.0027374434284865856,...    0.714912  
2  

{'answer': "I'm sorry, but I cannot provide a detailed answer without additional context or information on the paper's topic. The given embeddings include information on competing interests, author contributions, dietary control, experimental protocol, and an ischaemic pain test, but it is unclear what the paper is specifically about. Please provide more information or a specific query for me to summarize.",
 'sources': [{'Page 15': 'Competing Interests and Funding 511 No potential conflicts or competing interests are disclosed. No external funding was received.  512 Author Contrib...'},
  {'Page 6': 'Twenty-four hours prior to each trial, participants were instru cted to either restrict fluid intake 190 and avoid consumption of high-water content f...'},
  {'Page 22': '0246810\nEUH HYPO EUH HYPOVAS (cm)Early Follicular\nBaseline After water ingestion0246810\nEUH HYPO EUH HYPOMid-Luteal\nBaseline After water ingestion‡‡\n†...'}]}

In [82]:
query5 = """
Summarize the study's main take-aways in under 300 characters.
"""

reply5_5 = reply(df4, query5)
reply5_5

[{'Page 15': 'Competing Interests and Funding 511 No potential conflicts or competing interests are disclosed. No external funding was received.  512 Author Contrib...'}, {'Page 6': 'Twenty-four hours prior to each trial, participants were instru cted to either restrict fluid intake 190 and avoid consumption of high-water content f...'}, {'Page 4': 'METHODS 105 Ethical Approval 106 The study was approved by the Massey University Human Ethics Co mmittee: Southern A 107 (18/66). The study conformed ...'}]
                                                text  page  length  \
0  Competing Interests and Funding 511 No potenti...    15     614   
1  Twenty-four hours prior to each trial, partici...     6    4069   
2  METHODS 105 Ethical Approval 106 The study was...     4    3597   

                                          embeddings  similarity  
0  [-0.03124002367258072, -0.007716950494796038, ...    0.784409  
1  [-0.011679299175739288, 0.0027374434284865856,...    0.757329  
2  [-0.02

{'answer': 'The study investigated the effects of hypohydration and menstrual cycle phase on pain perception in healthy women. Participants were instructed to restrict fluid intake and avoid high-water content foods prior to the trial. The results suggest that hypohydration enhances pain perception during the early follicular phase of the menstrual cycle. No external funding was received, and all authors contributed to the work.',
 'sources': [{'Page 15': 'Competing Interests and Funding 511 No potential conflicts or competing interests are disclosed. No external funding was received.  512 Author Contrib...'},
  {'Page 6': 'Twenty-four hours prior to each trial, participants were instru cted to either restrict fluid intake 190 and avoid consumption of high-water content f...'},
  {'Page 4': 'METHODS 105 Ethical Approval 106 The study was approved by the Massey University Human Ethics Co mmittee: Southern A 107 (18/66). The study conformed ...'}]}

In [83]:
reply5_5['answer']

'The study investigated the effects of hypohydration and menstrual cycle phase on pain perception in healthy women. Participants were instructed to restrict fluid intake and avoid high-water content foods prior to the trial. The results suggest that hypohydration enhances pain perception during the early follicular phase of the menstrual cycle. No external funding was received, and all authors contributed to the work.'

In [85]:
query6 = """
Can you repeat what you just said?
"""

reply5_6 = reply(df4, query6)
reply5_6['answer']

[{'Page 6': 'Twenty-four hours prior to each trial, participants were instru cted to either restrict fluid intake 190 and avoid consumption of high-water content f...'}, {'Page 22': '0246810\nEUH HYPO EUH HYPOVAS (cm)Early Follicular\nBaseline After water ingestion0246810\nEUH HYPO EUH HYPOMid-Luteal\nBaseline After water ingestion‡‡\n†...'}, {'Page 23': '0246810\nEUH HYPO EUH HYPOVAS (cm)Early Follicular\nBaseline After water ingestion0246810\nEUH HYPO EUH HYPOMid-Luteal\nBaseline After water ingestion† ††...'}]
                                                text  page  length  \
0  Twenty-four hours prior to each trial, partici...     6    4069   
1  0246810\nEUH HYPO EUH HYPOVAS (cm)Early Follic...    22     153   
2  0246810\nEUH HYPO EUH HYPOVAS (cm)Early Follic...    23     154   

                                          embeddings  similarity  
0  [-0.011679299175739288, 0.0027374434284865856,...    0.721589  
1  [-0.013209676370024681, -0.011164473369717598,...    0.718228  

'I apologize, but I cannot provide a detailed answer without more context. The given embeddings are from two different sections of a scientific paper and do not provide enough information to understand the topic being discussed. Can you please provide more information or context regarding the paper and the question?'

In [86]:
chatbot

NameError: name 'chatbot' is not defined

# *End of Page*